1. Dataset Overview

In [2]:
import pandas as pd
import numpy as np
import pathlib
import csv
import re
from plotnine import *
from plotnine.data import *
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.cm import ScalarMappable


In [20]:

ds_games = pd.read_csv('steam_games.csv')

In [3]:

ds_users = pd.read_csv('steam-200k.csv')

In [22]:
ds_games

,url,types,name,desc_snippet,recent_reviews,all_reviews,release_date,developer,publisher,popular_tags,game_details,languages,achievements,genre,game_description,mature_content,minimum_requirements,recommended_requirements,original_price,discount_price
0,https://store.steampowered.com/app/379720/DOOM/,app,DOOM,Now includes all three premium DLC packs (Unto...,"Very Positive,(554),- 89% of the 554 user revi...","Very Positive,(42,550),- 92% of the 42,550 use...","May 12, 2016",id Software,"Bethesda Softworks,Bethesda Softworks","FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","Single-player,Multi-player,Co-op,Steam Achieve...","English,French,Italian,German,Spanish - Spain,...",54.0,Action,"About This Game Developed by id software, the...",NaN,"Minimum:,OS:,Windows 7/8.1/10 (64-bit versions...","Recommended:,OS:,Windows 7/8.1/10 (64-bit vers...",$19.99,$14.99
1,https://store.steampowered.com/app/578080/PLAY...,app,PLAYERUNKNOWN'S BATTLEGROUNDS,PLAYERUNKNOWN'S BATTLEGROUNDS is a battle roya...,"Mixed,(6,214),- 49% of the 6,214 user reviews ...","Mixed,(836,608),- 49% of the 836,608 user revi...","Dec 21, 2017",PUBG Corporation,"PUBG Corporation,PUBG Corporation","Survival,Shooter,Multiplayer,Battle Royale,PvP...","Multi-player,Online Multi-Player,Stats","English,Korean,Simplified Chinese,French,Germa...",37.0,"Action,Adventure,Massively Multiplayer",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,Mature Content Description The developers de...,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$29.99,NaN
2,https://store.steampowered.com/app/637090/BATT...,app,BATTLETECH,Take command of your own mercenary outfit of '...,"Mixed,(166),- 54% of the 166 user reviews in t...","Mostly Positive,(7,030),- 71% of the 7,030 use...","Apr 24, 2018",Harebrained Schemes,"Paradox Interactive,Paradox Interactive","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","Single-player,Multi-player,Online Multi-Player...","English,French,German,Russian",128.0,"Action,Adventure,Strategy",About This Game From original BATTLETECH/Mec...,NaN,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$39.99,NaN
3,https://store.steampowered.com/app/221100/DayZ/,app,DayZ,The post-soviet country of Chernarus is struck...,"Mixed,(932),- 57% of the 932 user reviews in t...","Mixed,(167,115),- 61% of the 167,115 user revi...","Dec 13, 2018",Bohemia Interactive,"Bohemia Interactive,Bohemia Interactive","Survival,Zombies,Open World,Multiplayer,PvP,Ma...","Multi-player,Online Multi-Player,Steam Worksho...","English,French,Italian,German,Spanish - Spain,...",NaN,"Action,Adventure,Massively Multiplayer",About This Game The post-soviet country of Ch...,NaN,"Minimum:,OS:,Windows 7/8.1 64-bit,Processor:,I...","Recommended:,OS:,Windows 10 64-bit,Processor:,...",$44.99,NaN
4,https://store.steampowered.com/app/8500/EVE_On...,app,EVE Online,EVE Online is a community-driven spaceship MMO...,"Mixed,(287),- 54% of the 287 user reviews in t...","Mostly Positive,(11,481),- 74% of the 11,481 u...","May 6, 2003",CCP,"CCP,CCP","Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","Multi-player,Online Multi-Player,MMO,Co-op,Onl...","English,German,Russian,French",NaN,"Action,Free to Play,Massively Multiplayer,RPG,...",About This Game,NaN,"Minimum:,OS:,Windows 7,Processor:,Intel Dual C...","Recommended:,OS:,Windows 10,Processor:,Intel i...",Free,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40828,https://store.steampowered.com/app/899836/Rock...,app,Rocksmith® 2014 Edition – Remastered – Sabaton...,NaN,NaN,NaN,"Feb 12, 2019",Ubisoft - San Francisco,NaN,"Casual,Simulation","Single-player,Shared/Split Screen,Downloadable...","English,German,French,Italian,Spanish - Spain,...",NaN,"Casual,Simulation","About This Content Play ""Ghost Division"" by S...",NaN,"Minimum:,OS:,Windows Vista, Windows 7, Windows...","Recommended:,OS:,Windows Vista, Windows 7,

In [23]:
ds_users

,151603712,The Elder Scrolls V Skyrim,purchase,1.0,0
0,151603712,The Elder Scrolls V Skyrim,play,273.0,0
1,151603712,Fallout 4,purchase,1.0,0
2,151603712,Fallout 4,play,87.0,0
3,151603712,Spore,purchase,1.0,0
4,151603712,Spore,play,14.9,0
...,...,...,...,...,...
199994,128470551,Titan Souls,play,1.5,0
199995,128470551,Grand Theft Auto Vice City,purchase,1.0,0
199996,128470551,Grand Theft Auto Vice City,play,1.5,0
199997,128470551,RUSH,purchase,1.0,0


2. Pre-processing of datasets

In [4]:
#data pre-processing
col_names=['user_id','game_title', 'behavior_name','value','temp'] 
ds_users = pd.read_csv("steam-200k.csv",header=None,names=col_names,
                    usecols=['user_id','game_title', 'behavior_name','value'])
#remove duplicated purchase
ds_users.drop_duplicates(subset=['user_id','game_title','behavior_name'], keep='first', inplace=True)
ds_users

,user_id,game_title,behavior_name,value
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0
1,151603712,The Elder Scrolls V Skyrim,play,273.0
2,151603712,Fallout 4,purchase,1.0
3,151603712,Fallout 4,play,87.0
4,151603712,Spore,purchase,1.0
...,...,...,...,...
199995,128470551,Titan Souls,play,1.5
199996,128470551,Grand Theft Auto Vice City,purchase,1.0
199997,128470551,Grand Theft Auto Vice City,play,1.5
199998,128470551,RUSH,purchase,1.0


In [25]:
#Number of unique users in user dataset
cond = ds_users.behavior_name == 'purchase'
ds_users[cond].user_id.nunique()

12393

In [26]:
#Number of unique games in user dataset
ds_users.game_title.nunique()


5155

In [6]:
#Reformatting user dataset to be used for content-based-recommender model
col_names=['user_id','game', 'purchase_play','hours', 'temp'] 
users = pd.read_csv("steam-200k.csv",header=None,names=col_names)
users.head()

,user_id,game,purchase_play,hours,temp
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0,0
1,151603712,The Elder Scrolls V Skyrim,play,273.0,0
2,151603712,Fallout 4,purchase,1.0,0
3,151603712,Fallout 4,play,87.0,0
4,151603712,Spore,purchase,1.0,0


In [7]:
users.drop('temp', inplace=True, axis=1)
new_users = users
new_users['purchase'] = new_users['purchase_play'] == 'purchase'
new_users['purchase'] = new_users['purchase'].astype(int)
new_users['play'] = new_users['purchase_play'] == 'play'
new_users['play'] = new_users['play'].astype(int)
new_users['hours'] = new_users['hours'] - new_users['purchase']
new_users = new_users.groupby(by=['user_id', 'game']).agg({'hours':'sum', 'purchase':'sum',
 'play':'sum'}).reset_index()
new_users

,user_id,game,hours,purchase,play
0,5250,Alien Swarm,4.9,1,1
1,5250,Cities Skylines,144.0,1,1
2,5250,Counter-Strike,0.0,1,0
3,5250,Counter-Strike Source,0.0,1,0
4,5250,Day of Defeat,0.0,1,0
...,...,...,...,...,...
128799,309626088,Age of Empires II HD Edition,6.7,1,1
128800,309812026,Counter-Strike Nexon Zombies,0.0,1,0
128801,309812026,Robocraft,0.0,1,0
128802,309824202,Dota 2,0.7,1,1


In [29]:
#Finally Outputting the new formatted user dataset to a new csv file
new_users.to_csv("steam_users_purchase_play.csv")

In [45]:

games = pd.read_csv("steam_games.csv", usecols=['name','genre', 'game_details','popular_tags', 'publisher', 'developer'])
users = pd.read_csv("steam_users_purchase_play.csv",
                    names=['user_id','game', 'hours','purchase', 'play'])
games['name'] = games['name'].fillna('')
#create column id for both the game and user dataset.
games["ID"] = ""
users["ID"] = ""


In [46]:
#Removing spaces and special characters from game names in both datasets
for i, row in games.iterrows():
    c = re.sub('[^A-Za-z0-9]+', '', row["name"])
    c = c.lower()
    games.at[i, 'ID'] = c

In [47]:
for i, row in users.iterrows():
    c = re.sub('[^A-Za-z0-9]+', '', row['game'])
    c = c.lower()
    users.at[i, 'ID'] = c

In [51]:
#finding all the games in games datasets that are present in user datasets
gameArr = users['ID'].unique()
print(len(gameArr))
uniqueGames = games['ID'].isin(gameArr)
commonGames = games[uniqueGames]
print(len(commonGames))

5152
3036


In [ ]:
#relevant information for recommendation purpose
#Cols focused are: genre, game_details, popular_tags, publisher, developer
commonGames.loc[:, 'genre'] = commonGames['genre'].fillna('')
commonGames.loc[:, 'game_details'] = commonGames['game_details'].fillna('')
commonGames.loc[:, 'popular_tags'] = commonGames['popular_tags'].fillna('')
commonGames.loc[:, 'publisher'] = commonGames['publisher'].fillna('')
commonGames.loc[:, 'developer'] = commonGames['developer'].fillna('')

In [53]:
#Function to remove spaces from column names to remove ambiguity in processing
def clean_data(x):
    if isinstance(x, str):
        return x.replace("", "")
    else:
        print(x)
        return x

In [ ]:
#Removing spaces between the words of columns of new Dataframe
commonGames.loc[:, 'genre'] = commonGames['genre'].apply(clean_data)
commonGames.loc[:, 'game_details'] = commonGames['game_details'].apply(clean_data)
commonGames.loc[:, 'popular_tags'] = commonGames['popular_tags'].apply(clean_data)
commonGames.loc[:, 'publisher'] = commonGames['publisher'].apply(clean_data)
commonGames.loc[:, 'developer'] = commonGames['developer'].apply(clean_data)

In [ ]:
#Creating new columns containing combinations of information for recommendations.
commonGames["genre_publisher_developer"] = commonGames['genre'] + commonGames['publisher'] + commonGames['developer']
commonGames["genre_popular_tags_developer"] = commonGames['genre'] + commonGames['popular_tags'] + commonGames['developer']
commonGames["genre_popular_tags_game_details"] = commonGames['genre'] + commonGames['popular_tags'] + commonGames['game_details']
commonGames["genre_publisher_developer_game_details"] = commonGames['genre'] + commonGames['publisher'] + commonGames['developer']+ commonGames['game_details']

In [57]:
commonGames.drop_duplicates('name')
commonGames.to_csv('processed_games_for_content-based.csv', index=False)

In [3]:
modified_gamesdata = pd.read_csv('processed_games_for_content-based.csv')
modified_gamesdata

,name,developer,publisher,popular_tags,game_details,genre,ID,genre_publisher_developer,genre_popular_tags_developer,genre_popular_tags_game_details,genre_publisher_developer_game_details
0,DayZ,Bohemia Interactive,"Bohemia Interactive,Bohemia Interactive","Survival,Zombies,Open World,Multiplayer,PvP,Ma...","Multi-player,Online Multi-Player,Steam Worksho...","Action,Adventure,Massively Multiplayer",dayz,"Action,Adventure,Massively MultiplayerBohemia ...","Action,Adventure,Massively MultiplayerSurvival...","Action,Adventure,Massively MultiplayerSurvival...","Action,Adventure,Massively MultiplayerBohemia ..."
1,EVE Online,CCP,"CCP,CCP","Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","Multi-player,Online Multi-Player,MMO,Co-op,Onl...","Action,Free to Play,Massively Multiplayer,RPG,...",eveonline,"Action,Free to Play,Massively Multiplayer,RPG,...","Action,Free to Play,Massively Multiplayer,RPG,...","Action,Free to Play,Massively Multiplayer,RPG,...","Action,Free to Play,Massively Multiplayer,RPG,..."
2,TERA,"Bluehole, Inc.","En Masse Entertainment,En Masse Entertainment","Free to Play,MMORPG,Massively Multiplayer,RPG,...","Multi-player,MMO,Co-op,Steam Trading Cards,Par...","Action,Adventure,Free to Play,Massively Multip...",tera,"Action,Adventure,Free to Play,Massively Multip...","Action,Adventure,Free to Play,Massively Multip...","Action,Adventure,Free to Play,Massively Multip...","Action,Adventure,Free to Play,Massively Multip..."
3,Stonehearth,Radiant Entertainment,"(none),(none)","City Builder,Building,Sandbox,Strategy,Surviva...","Single-player,Multi-player,Online Multi-Player...","Indie,Simulation,Strategy",stonehearth,"Indie,Simulation,Strategy(none),(none)Radiant ...","Indie,Simulation,StrategyCity Builder,Building...","Indie,Simulation,StrategyCity Builder,Building...","Indie,Simulation,Strategy(none),(none)Radiant ..."
4,Call of Duty®: Black Ops,Treyarch,"Activision,Activision","Action,FPS,Zombies,Multiplayer,Shooter,Singlep...","Single-player,Multi-player,Co-op,Steam Achieve...",Action,callofdutyblackops,"ActionActivision,ActivisionTreyarch","ActionAction,FPS,Zombies,Multiplayer,Shooter,S...","ActionAction,FPS,Zombies,Multiplayer,Shooter,S...","ActionActivision,ActivisionTreyarchSingle-play..."
...,...,...,...,...,...,...,...,...,...,...,...
3031,Black Ink,Bleank,"Bleank,Bleank","Design & Illustration,Utilities,Photo Editing,...",NaN,"Animation & Modeling,Design & Illustration,Pho...",blackink,"Animation & Modeling,Design & Illustration,Pho...","Animation & Modeling,Design & Illustration,Pho...","Animation & Modeling,Design & Illustration,Pho...","Animation & Modeling,Design & Illustration,Pho..."
3032,The Deer,Dodo Digital,"Dodo Digital,Dodo Digital","Gore,Adventure,Violent,Indie,Horror,Hunting,Fi...","Single-player,Steam is learning about this gam...","Adventure,Indie",thedeer,"Adventure,IndieDodo Digital,Dodo DigitalDodo D...","Adventure,IndieGore,Adventure,Violent,Indie,Ho...","Adventure,IndieGore,Adventure,Violent,Indie,Ho...","Adventure,IndieDodo Digital,Dodo DigitalDodo D..."
3033,Tank Operations: European Campaign,Linked Dimensions,"rokapublish,rokapublish","Early Access,Strategy,World War II,Military,Wa...","Single-player,Steam Achievements,Steam is lear...","Strategy,Early Access",tankoperationseuropeancampaign,"Strategy,Early Accessrokapublish,rokapublishLi...","Strategy,Early AccessEarly Access,Strategy,Wor...","Strategy,Early AccessEarly Access,Strategy,Wor...","Strategy,Early Accessrokapublish,rokapublishLi..."
3034,King-Dom,maddma,"maddma,maddma","Strategy,Chess,Indie","Online Multi-Player,Steam Achievements,Steam W...",Strategy,kingdom,"Strategymaddma,maddmamaddma","StrategyStrategy,Chess,Indiemaddma","StrategyStrategy,Chess,IndieOnline Multi-Playe...","Strategymaddma,maddmamaddmaOnline Multi-Player..."
